In [2]:
# get imports # import files
from threading import Thread
import queue as que
import pandas as pd
import os, pickle
import nltk
import math
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
from collections import Counter 

stemmer = SnowballStemmer("english")
cachedStopWords = stopwords.words("english")

In [3]:
#methods
def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def seperator(docs):
    print(len(docs))
    txt = []
    typesInfo = []
    docsInfo = []
    for line in docs:
        words = line.split()
        content = (" ").join(words[1:])
        item = (", ").join([words[0], content])
        txt.append(item)
        typesInfo.append(words[0])
        docsInfo.append(content)
    return txt, typesInfo, docsInfo

def write(data):
    print(type(data))
    with open('./training_data.csv', 'w') as f:
        for item in data:
            f.write("%s\n" % item)
            
# methods
def read(fileName):
    with open(fileName) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    return content

def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def vocabDict(vocab, n): # handles empty word sets
    nk = 0
    # n = the number of word postions for this document type
    probability = ((nk + 1) / (n + len(vocab)))

    newDict = {"count": 0, "probability":probability}
    vdict = dict((el,newDict) for el in vocab)
    return vdict

def wordCount(doc, vocab, n): # get actual word count for each word and create dict frame to track the info
    counts = dict(Counter(doc.tolist()[0].split()))
    for key, value in counts.items():
        nk = value
        probability = ((nk + 1) / (n + len(vocab)))
        newDict = {"count": nk, "probability": probability}
        counts[key] = newDict
    emptyVocab = vocabDict(vocab, n)
    dicts = [merge_two_dicts(emptyVocab, counts)]
    return dicts
    

def saveFrame(df, name):
    df.to_csv(name+".csv", index=False, sep=",", header=True)
    save_it_all(df, name+".pkl")

In [4]:
# # load in files 
# # read file line by line 
# fileName = "../forumTraining.data"
# data = read(fileName)
# len(data)
# sep_data, typesInfo, docsInfo = seperator(data)
# # write(sep_data) not needed the raw csv it makes IS BROKEN
# # print(len(list(set(typesInfo))))
# # print(list(set(typesInfo)))
# dataFrameRaw = pd.DataFrame({"Type": typesInfo, "Document": docsInfo})
# saveFrame(dataFrameRaw, "raw_training_data.csv")

In [5]:
raw_training_data = pd.read_csv("../RawFiles/raw_training_data.csv", sep=",")
raw_training_data.head()

,Type,Document
0,atheism,alt atheism faq atheist resources archive name...
1,atheism,alt atheism faq introduction to atheism archiv...
2,atheism,re gospel dating in article mimsy umd edu mang...
3,atheism,re university violating separation of church s...
4,atheism,re soc motss et al princeton axes matching fun...


### Trianing data raw is held in the dataframe "raw_training_data"

In [6]:
# create the raw vocabulary set:
raw_vocabulary = " ".join(raw_training_data["Document"].tolist()) # concatinates all documents in the data frame
raw_vocabulary = list(set(raw_vocabulary.split()))
print(len(raw_vocabulary))
# save_it_all(raw_vocabulary, "./raw_vocabulary.pkl")

73712


### The raw vocabulary is saved in the above cell and refrenced as "raw_vocabulary" 

# This is the learning step: make it dynamic!
For each class cj (document type) in C
    
    1. Docsj ← training documents for which the classification is cj
    
    2. Probability estimate of a particular class: P(cj) = |Docsj| / |training documents|
    
    3. Textj ← create a single document per class (concatenate all Docsj)
    
    4. n = total number of word positions in Textj
    
    5. For each word wk in Vocabulary nk = number of times wk occurs in Textj
    
    6. Estimate of word occurrence for particular document type: P(wk | cj) = (nk + 1) / (n + |Vocabulary|)
    
Probability of kth word in vocabulary, given a document of type j

In [7]:
# load in docs you want to use:
## trainign data set for use:
trainFrame = pd.read_csv("../RawFiles/raw_training_data.csv", sep=",")

## vocabulary training set for use:
vocabulary = load_objects("../RawFiles/raw_vocabulary.pkl")

# validate:
print(type(vocabulary))
trainFrame.head()

<class 'list'>


,Type,Document
0,atheism,alt atheism faq atheist resources archive name...
1,atheism,alt atheism faq introduction to atheism archiv...
2,atheism,re gospel dating in article mimsy umd edu mang...
3,atheism,re university violating separation of church s...
4,atheism,re soc motss et al princeton axes matching fun...


In [8]:
# single docs per class
types = list(set(trainFrame["Type"]))
singleFrame = pd.DataFrame({"Type":types , "Document": None})
# singleFrame.head(20)
for docType in singleFrame["Type"]:
    #print(trainFrame["Document"][trainFrame["Type"]==docType])
    singleFrame["Document"][singleFrame["Type"]== docType] = " ".join(trainFrame["Document"][trainFrame["Type"]==docType].tolist())
singleFrame.head(30)
singleFrame["Type"].tolist()


['cryptology',
 'graphics',
 'autos',
 'electronics',
 'baseball',
 'atheism',
 'forsale',
 'space',
 'politics',
 'mac',
 'xwindows',
 'hockey',
 'mideastpolitics',
 'religion',
 'pc',
 'medicine',
 'guns',
 'christianity',
 'motorcycles',
 'mswindows']

In [10]:
# make a copy of the single frame 
rawDataInfoFrame = singleFrame.copy()

In [11]:
# probability estimate of the class
rawDataInfoFrame["ClassProbability"] = None
for docType in rawDataInfoFrame["Type"]:
    rawDataInfoFrame["ClassProbability"][rawDataInfoFrame["Type"]== docType] = len(trainFrame["Document"][trainFrame["Type"]== docType].tolist()) / trainFrame["Type"].count()


In [12]:
rawDataInfoFrame.head()

,Type,Document,ClassProbability
0,cryptology,ripem frequently asked questions archive name ...,0.0526875
1,graphics,call for presentations navy sciviz vr seminar ...,0.0517135
2,autos,re saturn s pricing policy in article c vir l ...,0.052599
3,electronics,re help with tracking device in article fba e ...,0.0523333
4,baseball,spring records the orioles pitching staff agai...,0.0528646


In [13]:
# get the word postions:
rawDataInfoFrame["WordPositions"] = None
for docType in rawDataInfoFrame["Type"]:
    rawDataInfoFrame["WordPositions"][rawDataInfoFrame["Type"]== docType] = len(rawDataInfoFrame["Document"][rawDataInfoFrame["Type"]== docType].tolist()[0].split())
    


In [23]:
rawDataInfoFrame.head(20)

,Type,Document,ClassProbability,WordPositions
0,medicine,info needed gaucher s disease i have a yr old ...,0.052599,170336
1,christianity,re sex education in article mar athos rutgers ...,0.0529532,216581
2,xwindows,th international obfuscated c code contest ope...,0.0525104,175024
3,autos,re saturn s pricing policy in article c vir l ...,0.052599,127696
4,mac,re nutek faces apple s wrath article read in a...,0.0511821,97182
5,graphics,call for presentations navy sciviz vr seminar ...,0.0517135,123428
6,atheism,alt atheism faq atheist resources archive name...,0.0425042,161442
7,forsale,motorcycle wanted followup to kedz wpi wpi edu...,0.051802,69761
8,pc,wanted bus card for logitech mouse i ve acquir...,0.0522448,112583
9,hockey,superstars and attendance was teemu selanne wa...,0.0531303,153949


In [24]:
# word count 
rawDataInfoFrame["WordCount"] = None
for docType in rawDataInfoFrame["Type"]:
    n = rawDataInfoFrame["WordPositions"][rawDataInfoFrame["Type"] == docType].tolist()[0]
    print(n)
    rawDataInfoFrame["WordCount"][rawDataInfoFrame["Type"] == docType] = wordCount(rawDataInfoFrame["Document"][rawDataInfoFrame["Type"]==docType], vocabulary, n)
    

170336
216581
175024
127696
97182
123428
161442
69761
112583
153949
95238
119407
272488
202625
191723
129611
115503
219507
115430
168481


In [25]:
rawDataInfoFrame.head()
#rawDataInfoFrame["WordCount"][rawDataInfoFrame["Type"] == "graphics"].tolist()[0]

,Type,Document,ClassProbability,WordPositions,WordCount
0,medicine,info needed gaucher s disease i have a yr old ...,0.052599,170336,"{'hepis': {'count': 0, 'probability': 4.097554..."
1,christianity,re sex education in article mar athos rutgers ...,0.0529532,216581,"{'hepis': {'count': 1, 'probability': 6.889590..."
2,xwindows,th international obfuscated c code contest ope...,0.0525104,175024,"{'hepis': {'count': 0, 'probability': 4.020326..."
3,autos,re saturn s pricing policy in article c vir l ...,0.052599,127696,"{'hepis': {'count': 0, 'probability': 4.965046..."
4,mac,re nutek faces apple s wrath article read in a...,0.0511821,97182,"{'hepis': {'count': 0, 'probability': 5.851580..."


In [26]:
saveFrame(rawDataInfoFrame, "./raw_data_info_frame")

### Removeing words less than 3 in length words from the stuff

In [15]:
## get a vocab set of removed stop words and the train frame with removed stop words
# remove stop words
word_len_training_data = singleFrame.copy()
for docType in word_len_training_data["Type"]:
    text = word_len_training_data["Document"][word_len_training_data["Type"]==docType].tolist()[0]
    word_len_training_data["Document"][word_len_training_data["Type"]==docType] = ' '.join([word for word in text.split() if len(word) > 3])

word_len_vocabulary = []
word_len_vocabulary = set([word for word in raw_vocabulary if len(word) > 3])
len(word_len_vocabulary)

len(word_len_vocabulary)
#word_len_training_data.head()


68203

In [17]:
# probability estimate of the class
word_len_training_data["ClassProbability"] = None
for docType in word_len_training_data["Type"]:
    word_len_training_data["ClassProbability"][word_len_training_data["Type"]== docType] = len(trainFrame["Document"][trainFrame["Type"]== docType].tolist()) / trainFrame["Type"].count()


In [18]:
word_len_training_data.head()

,Type,Document,ClassProbability
0,cryptology,ripem frequently asked questions archive name ...,0.0526875
1,graphics,call presentations navy sciviz seminar call pr...,0.0517135
2,autos,saturn pricing policy article shuksan boeing f...,0.052599
3,electronics,help with tracking device article aesop rutger...,0.0523333
4,baseball,spring records orioles pitching staff again ha...,0.0528646


In [19]:
# get the word postions:
word_len_training_data["WordPositions"] = None
for docType in word_len_training_data["Type"]:
    word_len_training_data["WordPositions"][word_len_training_data["Type"]== docType] = len(word_len_training_data["Document"][word_len_training_data["Type"]== docType].tolist()[0].split())
   

In [20]:
word_len_training_data.head()

,Type,Document,ClassProbability,WordPositions
0,cryptology,ripem frequently asked questions archive name ...,0.0526875,124900
1,graphics,call presentations navy sciviz seminar call pr...,0.0517135,70311
2,autos,saturn pricing policy article shuksan boeing f...,0.052599,69565
3,electronics,help with tracking device article aesop rutger...,0.0523333,63641
4,baseball,spring records orioles pitching staff again ha...,0.0528646,67041


In [23]:
# word count 
word_len_training_data["WordCount"] = None
for docType in word_len_training_data["Type"]:
    n = word_len_training_data["WordPositions"][word_len_training_data["Type"] == docType].tolist()[0]
    print(n)
    word_len_training_data["WordCount"][word_len_training_data["Type"] == docType] = wordCount(word_len_training_data["Document"][word_len_training_data["Type"]==docType], word_len_vocabulary, n)
   

124900
70311
69565
63641
67041
89590
41823
99875
114319
52749
96182
88718
158062
72400
59549
99326
109058
118932
62027
51748


In [27]:
word_len_training_data["WordCount"][word_len_training_data["Type"]=="autos"].tolist()

[{'seinfeldsupersonicsrushbelafleck': {'count': 0,
   'probability': 7.258579641135822e-06},
  'raods': {'count': 2, 'probability': 2.1775738923407468e-05},
  'gonzalz': {'count': 0, 'probability': 7.258579641135822e-06},
  'somethingness': {'count': 0, 'probability': 7.258579641135822e-06},
  'algorhtyms': {'count': 0, 'probability': 7.258579641135822e-06},
  'examines': {'count': 0, 'probability': 7.258579641135822e-06},
  'outpouring': {'count': 0, 'probability': 7.258579641135822e-06},
  'http': {'count': 0, 'probability': 7.258579641135822e-06},
  'bobml': {'count': 3, 'probability': 2.903431856454329e-05},
  'bogs': {'count': 1, 'probability': 1.4517159282271644e-05},
  'psuedocolor': {'count': 0, 'probability': 7.258579641135822e-06},
  'israelis': {'count': 0, 'probability': 7.258579641135822e-06},
  'ingrina': {'count': 0, 'probability': 7.258579641135822e-06},
  'electrophoresis': {'count': 0, 'probability': 7.258579641135822e-06},
  'rowlands': {'count': 0, 'probability': 7.

In [29]:
saveFrame(word_len_training_data, "./word_len_training_data")
save_it_all(word_len_vocabulary, "./word_len_vocabulary.pkl")